# Visualize artifacts and the detections on them 

### Setup main notebook options

In [1]:
region_name = "TDS"  #_ELAISN1

# Add artifacts here
dd_arts = [{"field_id":"GNU2597312684324028416", "ra":243.31, "dec":55.79, "rad":0.01},   # 0 Detection structures next to bright source
           {"field_id":"GNU2597312684324028416", "ra":243.28, "dec":55.76, "rad":0.02},   # 1 Multiple circular artifacts due to bright star
           {"field_id":"GNU2597312684324028416", "ra":243.41, "dec":56.27, "rad":0.02},   # 2 Edge line
           {"field_id":"GNU2597523790556561408", "ra":241.405, "dec":55.615, "rad":0.01}, # 3 Ring without bright source,  Galaxy MCG+09-26-052 
           {"field_id":"GNU2597523790556561408", "ra":241.58, "dec":55.43, "rad":0.01},   # 4 Line artifact
           {"field_id":"GNU2597523790556561408", "ra":241.77, "dec":55.48, "rad":0.05},   # 5 Multiple rings
           {"field_id":"GNU2597523790556561408", "ra":240.81, "dec":55.17, "rad":0.03},   # 6 Faint ring
           {"field_id":"GNU2597594159300739072", "ra":244.30, "dec":53.695, "rad":0.02},  # 7 Edge line
           {"field_id":"GNU2597664528044916736", "ra":242.79, "dec":53.97, "rad":0.01},   # 8 Ghost next to bright source
           {"field_id":"GNU2597664528044916736", "ra":242.77, "dec":54.36, "rad":0.03},   # 9 Faint ring
           {"field_id":"GFU2597312684324028416", "ra":243.36, "dec":55.75, "rad":0.07},   # 10 Very bright star makes cross, also effects in NUV
           {"field_id":"GNU2597383053068206080", "ra":245.22, "dec":54.90, "rad":0.03},   # 11 Ring
           {"field_id":"GNU2597734896789094400", "ra":241.67, "dec":54.69, "rad":0.03},   # 12 Ring
           {"field_id":"GNU2597734896789094400", "ra":241.12, "dec":54.42, "rad":0.03},   # 13 Ring
           {"field_id":"GFU2597523790556561408", "ra":241.405, "dec":55.615, "rad":0.03},  # 14 Ring without bright source, also NuV, Galaxy MCG+09-26-052
           # From here on TDS non TDS_ELAI artifacts
           {"field_id":"GNU2592738715952480256", "ra":34.92, "dec":-4.35, "rad":0.08},   #15 Large circle of circles with multiple detections on it
           #{"field_id":"", "ra":, "dec":, "rad":0.01}, 
           #{"field_id":"", "ra":, "dec":, "rad":0.01}, 
           #{"field_id":"", "ra":, "dec":, "rad":0.01}, 
        ]

# Variables to select detections and plotting options
import numpy as np

#Select artifacts that lead to detection removal. New column with "art_sel" will be created for detections to cut on. 
#This variable is zero if none of the artifacts below is selected, othervise the "artifact value"
art_bit_sel = [1,2,4,8,16,32,64,128,256,512] # [2,4,8,128]

# http://www.galex.caltech.edu/wiki/GCAT_Manual#Flags
[1,2,4,8,16,32,64,128,256,512]
#    ( 1): edge; Detector bevel edge reflection (NUV only).
#    ( 2): window; Detector window reflection (NUV only).
#    ( 4): dichroic; Dichroic reflection.
#    ( 8): varpix; Varible pixel based on time slices.
#    ( 16): brtedge; Bright star near field edge (NUV only)
#    ( 32): Detector rim(annulus); proximity(>0.6 deg fld ctr)
#    ( 64): dimask; dichroic reflection artifact mask flag
#    (128): varmask; Masked pixel determined by varpix.
#    (256): hotmask; Detector hot spots.
#    (512): yaghost; Possible ghost image from YA slope. This is only relevant for data taken after the completion of GR6. 

s2n_min    = 5                                                       # Minimum s2n of detections
r_fov_max  = 0.5                                                     #Maximum distance from the camera center
vars       = ["ellip_world","size_world","class_star","art_sel"]     # Variables to be used to select detections on artifacts
vars_sel   = [[0.25,1.0]    ,[6,np.inf] ,[-0.5,0.1] ,[0.5,np.inf]]   # Selection of detections on artifacts
vars_col   = ["b"          ,"m"         ,"g"         ,"w"]           # Circle color for selected detections on artifacts
vars_ms    = [8            ,6.5         , 5          ,3.5]           # Marker size of circles showing detections selected by these artifacts. 
vars_bins  = [40           ,"sqrt"      , 40         ,"sqrt"]        # Nr bins for histogram plots
vars_log   = [False        ,True        , False      , True ]        # Show histograms in logy?
lw = 0.8                                                             # Line width for circles of detections on skymaps

### Load region

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
from astropy.table import Table
from astropy.nddata import bitmask

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux, sel_sources, dd_filter2id

#Load region and field
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
rg = Region()
rg.load_from_fits(region_fname)

### Display detections

In [3]:
def plot_art(art_idx):
    # Get field
    dd_art = dd_arts[art_idx]
    fd = rg.get_field(field_id=dd_art["field_id"], load_method="FITS", add_field=True)
    
    # Prepare figute and sky region
    fig_art = plt.figure("Sky map artifact",figsize=(8, 7))
    sky_center = SkyCoord(dd_art["ra"]*uu.deg, dd_art["dec"]*uu.deg, frame="icrs")
    sky_art= CircleSkyRegion(center=sky_center, radius=dd_art["rad"]*uu.deg)
    plt.cla()
    ax, wcs_art = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = sky_art, fig=fig_art)

    #Apply s2n  and r_fov cutcut
    nr_det = len(fd.tt_detections)
    sel_s2n = (fd.tt_detections["s2n"]>s2n_min)
    sel_fov = (fd.tt_detections["r_fov"]<r_fov_max)
    sel_s2nfov = sel_s2n * sel_fov
    fd.tt_detections["sel"]  = sel_s2nfov
    tt_det_all = fd.tt_detections[sel_s2nfov]


    # Apply bitmask cut
    # See https://docs.astropy.org/en/stable/api/astropy.nddata.bitfield_to_boolean_mask.html
    bit = bitmask.bitfield_to_boolean_mask(
        tt_det_all["artifacts"],
        flip_bits=True,
        dtype=bool,
        good_mask_value=0,
        ignore_flags=art_bit_sel,
    )
    tt_det_all["art_sel"] = tt_det_all["artifacts"] * bit

    #Plot all detections that pass s2n
    kdet_s2n = {"marker":"o","color":"r","markersize": 10,"alpha":1,"markeredgewidth": lw}
    ax, tt_det = vvis.plot_sky_sources(tt_det_all, src_id=None,src_kwargs=kdet_s2n, sky_region_wcs = (sky_art,wcs_art), only_selected=True)
    
    if len(tt_det) == 0:
        print("No detections passed the s2n and r_fov cut in this cutout")
        plt.show()
        return

    #Get cut efficiencies for field
    tt_eff = Table()
    tt_eff["s2n"] = [round(sel_s2n.sum()/nr_det,6)]
    tt_eff["r_fov"] = [round(sel_fov.sum()/nr_det,6)]
    tt_eff["s2nfov"] = [round(sel_s2nfov.sum()/nr_det,6)]
    sel_art_tot = np.zeros(len(tt_det_all), dtype=bool) # Total selections without artifactcut
    for ii in range(len(vars)):
        sel_art = (tt_det_all[vars[ii]]>vars_sel[ii][0]) * (tt_det_all[vars[ii]]<vars_sel[ii][1])
        sel_art_tot += sel_art
        tt_eff[vars[ii]] = [round(1.-sel_art.sum()/ nr_det,6)]
    sel_noart = ~(sel_art_tot>0)
    tt_eff["total"] = [round((sel_s2nfov.sum()/ nr_det)*(sel_noart.sum()/len(tt_det_all)),6)]

    # Add s2n und fov cut efficiencies
    
    #Plot circles for detections passing different cuts
    for ii in range(len(vars)):
        sel= (tt_det[vars[ii]]>vars_sel[ii][0]) * (tt_det[vars[ii]]<vars_sel[ii][1])
        kdet_ell = {"marker":"o","color":vars_col[ii],"markersize": vars_ms[ii],"alpha":1,"markeredgewidth": lw}
        vvis.plot_sky_sources(tt_det[sel], src_id=None,src_kwargs=kdet_ell, sky_region_wcs = (sky_art,wcs_art), only_selected=True)
    plt.tight_layout()
    plt.show()
    
    #Draw some distributions in field
    fig_dist, axs = plt.subplots(1,4,figsize=(12, 3.5))
    axs = axs.flatten()
    for ii in range(0,4):
        axs[ii].hist([tt_det_all[vars[ii]],tt_det[vars[ii]]],density=True, bins = vars_bins[ii], log = vars_log[ii], histtype="step")
        axs[ii].set_xlabel(vars[ii])
    plt.tight_layout()
    plt.show()

    display(tt_det[["ra","dec", "artifacts","r_fov","s2n"]+vars])
    display("Kept fraction of detections in entire field",tt_eff)
widgets.interactive( plot_art, art_idx=widgets.IntSlider(min=0, max=len(dd_arts)-1, value=0, step=1))

interactive(children=(IntSlider(value=0, description='art_idx', max=15), Output()), _dom_classes=('widget-inte…